In [2]:
#creating spark session

In [2]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .master("local[*]")
    .appName("Spark Streaming")
    .config("spark.driver.host","localhost")
    .config("spark.driver.port","9999")
    .getOrCreate()
)

In [3]:
spark

In [5]:
#read input data

df_raw = spark.readStream.format("socket") \
.option("host","localhost") \
.option("port","9999") \
.load()

In [6]:
#splite the line into words
from pyspark.sql.functions import split

df_words = df_raw.withColumn("words", split("value",' '))

In [8]:
#explod the list of words
from pyspark.sql.functions import explode

df_explode = df_words.withColumn("word",explode("words")).drop("value","words")

In [9]:
#aggregate the words to generate count
from pyspark.sql.functions import count,lit

df_agg = df_explode.groupBy("word").agg(count(lit(1)).alias("cnt"))

In [ ]:
from pyspark.sql.utils import StreamingQueryException

try:
    query = df_agg.writeStream \
        .format("console") \
        .outputMode("complete") \
        .start()
    
    query.awaitTermination()

except StreamingQueryException as e:
    print(f"StreamingQueryException: {e}")

finally:
    print("Ending the streaming process.")
